In [ ]:
#Libraries
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import sys

#Create the credentials object
keyfile_location = 'client_secrets.json'
scopes = ['https://www.googleapis.com/auth/analytics.readonly','https://www.googleapis.com/auth/analytics.edit']
credentials = ServiceAccountCredentials.from_json_keyfile_name(keyfile_location, scopes=scopes)

#Build a service object
api_name = 'analytics'
api_version = 'v3'
service = build(api_name, api_version, credentials=credentials)

#Call the API for things
#Grabs account details that are available for the credentialed user
account_summary = service.management().accountSummaries().list().execute()

#Get accounts that are returned from the list method
accounts = account_summary.get('items')

#Define list structures for what we'll grab out of accounts
account_ids = []
properties = []
property_ids = []
property_names = []

#Attempting to iterate over lists/dictionaries in accounts
for a in range(len(accounts)):
    account_ids.append(accounts[a].get('id'))
    properties.append(accounts[a].get('webProperties'))
    #print('Account loop: ', a, 'Account ID: ', accounts[a].get('id'))
    
    for p in range(len(properties)):
        #print('Property Loop:', p)
       
        for i in range(len(properties[p])):
            property_ids.append(properties[p][i].get('id'))
            property_names.append(properties[p][i].get('name'))
            #print('ID loop: ', i, 'Account ID: ', accounts[a].get('id'), '-', 'Property ID: ', properties[p][i].get('id'), '\n')

#This removes duplicate properties from list
property_ids = list(set(property_ids))
property_names = list(set(property_names))

print('Service and account object built.\n')
print('Relevant functions:')
print('checkProperties()')
print('checkDimensions()')
print('singleDimension()')
print('multiDimension()')

In [ ]:
#Functions

#Displays what the user has access to
def checkProperties():
    #Verifies we have what we think for accounts and properties
    print('Account IDs Array: %s' % account_ids)
    print('Property IDs Array: %s' % property_ids)
    print('Property Names Array: %s' % property_names)

#Prints dimension details
def printDimensions():
    print('Dimension Kind = %s' % dimension.get('kind'))
    print('Dimension Id = %s' % dimension.get('id'))
    print('Account ID = %s' % dimension.get('accountId'))
    print('Property ID = %s' % dimension.get('webPropertyId'))
    print('Dimension Name = %s' % dimension.get('name'))
    print('Dimension Index = %s' % dimension.get('index'))
    print('Dimension Scope = %s' % dimension.get('scope'))
    print('Dimension Active = %s' % dimension.get('active'))
    print('Dimension Created = %s' % dimension.get('created'))
    print('Dimension Updated = %s' % dimension.get('updated'))
    print('\n')

#Grabs and displays all dimensions by account, by property

#THIS WAS BREAKING - Says items aren't defined. Also, you shoul all this to run with nothing

def checkDimensions():

    #Allows for input of which dimensions to get and store
    dimension_values = input('Which Scope, Index, Property, or Account? --->')

    #Runs through each property ID and grabs account ID, prints details
    for p in range(len(property_ids)):
        account_id = property_ids[p].split('-', )

        #For each property in the account grabs the dimensions
        dimensions = service.management().customDimensions().list(
            accountId = account_id[1],
            webPropertyId = property_ids[p],
        ).execute()

        #Prints out custom dimensions in the property
        if not dimension_values:
            for dimension in dimensions.get('items', []):
                printDimensions()

        elif dimension_values:
            for dimension in dimensions.get('items', []):
                while dimension_values == dimension.get('scope'):
                    printDimensions()
                    break
                while dimension_values == dimension.get('index'):
                    printDimensions()
                    break
                while dimension_values == dimension.get('webPropertyId'):
                    printDimensions()
                    break
                while dimension_values == dimension.get('accountId'):
                    printDimensions()
                    break
                while dimension_values == dimension.get('id'):
                    printDimensions()
                    break

        print('----------END OF %s----------\n' % property_names[p])
    
#Inserts a single custom dimension as specified
def singleDimension():
    
    #Input account details
    web_property_id = input('Enter valid format UA tracker, e.g. UA-XXXXXXXX-X --->')
    account_id = web_property_id.split('-', )[1]
    
    #Input dimension details
    dimension_name = input('Enter text name for new dimension --->')
    dimension_scope = input('Enter HIT, SESSION, or USER scope --->')
    dimension_status = input('Enter True for active and leave blank for inactive --->')
    
    try:
        #Use management API insert method
        service.management().customDimensions().insert(
              accountId = account_id,
              webPropertyId = web_property_id,
              body = {
                  'name': dimension_name,
                  'scope': dimension_scope,
                  'active': dimension_status
              }
        ).execute()
        print("%s was created in" % dimension_name, web_property_id)
    
    except:
        print("Following error occured:", sys.exc_info()[0])
    
#Creates defined dimension for all properties
def multiDimension():

    #Dimension details to be created, list length must match
    dimension_names = input('Enter comma separated list of dimension names without spaces between commas, e.g. "First input,Second Input,Third Input" --->').split(',' )
    dimension_scopes = input('Enter comma separated list of corresponding dimension scopes without spaces between commas, e.g. "SESSION,HIT,SESSION" --->').split(',' )
    dimension_states = input('Enter comma separated list of corresponding dimension status without spaces between commas, e.g. "True,False,True" --->').split(',' )

    #Runs through states list to set values to boolean
    for i in range(len(dimension_states)):
        if dimension_states[i] == 'True':
            dimension_states.pop(i)
            dimension_states.insert(i, True)
        elif dimension_states[i] == 'False':
            dimension_states.pop(i)
            dimension_states.insert(i, False)

    #Ensures all lists are equivalent in length before running insert dimension
    if len(dimension_names) == len(dimension_scopes) == len(dimension_states):
        print("Number of dimensions are equivalent and will be inserted.")
        
        try:
            #Runs through each property ID and grabs account ID
            for p in range(len(property_ids)):
                account_id = property_ids[p].split('-', )

                #For each property runs through each dimension to be inserted
                for d in range(len(dimension_names)):
                    service.management().customDimensions().insert(
                        accountId = account_id[1],
                        webPropertyId = property_ids[p],
                        body = {
                            'name': dimension_names[d],
                            'scope': dimension_scopes[d],
                            'active': dimension_states[d]
                        }
                    ).execute()
                    print("%s was created in" % dimension_names[d], property_ids[p])
        except:
            print("Following unexpected error occurred:", sys.exc_info()[0])
                
    else:
        print("Inputs are inequivalent and won't be inserted.")